# Using dask distributed cluster to speed up computation of PV potential by factor of 10

In [1]:
import xarray as xr
import numpy as np
import time
from dask.distributed import Client
import core as core

import warnings
warnings.filterwarnings("ignore")

In [2]:
# location of era5 data on teachinghub
path="/home/voigta80/LEHRE/msc-intro-comp-met-ex-w2024/data/era5/"

Start dask cluster with 10 processes (workers), each with 5 threads. 

In [3]:
client = Client(n_workers=20, threads_per_worker=5, loca_directory="/tmp/tmpvoigta80")

2024-10-30 15:11:47,077 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-09ucm0a1', purging
2024-10-30 15:11:47,078 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8hnk8l9k', purging
2024-10-30 15:11:47,078 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-y69gtjzm', purging
2024-10-30 15:11:47,078 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8in2wy56', purging
2024-10-30 15:11:47,078 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-rag2a5dc', purging
2024-10-30 15:11:47,079 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-kg1uphkq', purging
2024-10-30 15:11:47,079 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-52uy9g3g', purging

In [4]:
client

<Client: 'tcp://127.0.0.1:44847' processes=20 threads=100, memory=753.83 GiB>

In [5]:
start_time = time.time()
ds=xr.open_mfdataset(path+"era5-2000-*.nc", engine="netcdf4", chunks={"valid_time":1e5} )
ds["wspd"] = core.windspeed(ds)
pvpot = core.pv_pot(ds).groupby(ds.valid_time.dt.month).mean("valid_time").compute()
end_time = time.time()
execution_time = end_time - start_time

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Ao

In [6]:
print(f"Execution time: {execution_time:.5f} seconds")

Execution time: 78.72001 seconds


In [7]:
client.shutdown()

In [8]:
pvpot

<xarray.DataArray (month: 12, latitude: 721, longitude: 1440)> Size: 50MB
array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.48196262, 0.48198524, 0.4820098 , ..., 0.48184314,
         0.48187613, 0.4819154 ],
        [0.4816251 , 0.48164183, 0.48165774, ..., 0.4815678 ,
         0.48158753, 0.4816064 ],
        [0.4679344 , 0.4679344 , 0.4679344 , ..., 0.4679344 ,
         0.4679344 , 0.4679344 ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
...
        [0.41012865, 0.41013625, 0.41014126, ..., 0.4100696 ,
         0.41008776, 0.41010714],
        [0.40905693, 0.40906295, 0.40906733, ..., 0.40901965,
         0.409032  , 0.40904516],
        [0.3937947 , 0.3937947 , 0.3937947 , ..., 0.3937947 ,
         0.3937947 , 0.3937947 ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.55768853, 0.55769956, 0.55770415, ..., 0.557582  ,
         0.5576082 , 0.55764294],
        [0.55730903, 0.5573125 , 0.55731565, ..., 0.55726105,
         0.5572768 , 0.55729276],
        [0.54138875, 0.54138875, 0.54138875, ..., 0.54138875,
         0.54138875, 0.54138875]]], dtype=float32)
Coordinates:
    number     int64 8B 0
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12